In [1]:

import requests
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml,html5lib
import time
import locale


In [2]:
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
prices_list=[]#list of prices

In [3]:
def avg_price(prices,length):
   
    flag=5#number of iterations to make an average price from list.
    counter=0
    sum=0
    
    if length<=5:
        flag=length
        
    for i in range(0,flag):
        
        num_len=len(prices[i].get_text().strip()[1:])#to validate that game really have a value. (problem with web-site used)
        
        if num_len<=0:
            continue
        
        else:
            if prices[i] is None:
                continue
            else:
                price=locale.atof(prices[i].get_text().strip()[1:])
                #print(price)
        
            if price > 100:
                continue
            
            else:
                sum+=price
                counter+=1
        return float(sum/counter)

In [4]:
def scrape_each_game_price(name,name_for_search,game_year):
    #time.sleep(5)
    url='https://www.pricecharting.com/search-products?type=prices&q='+name_for_search+'&go=Go'
    userAgent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=userAgent)
    soup = BeautifulSoup(response.text,"html.parser")

    #names=soup.find_all('td',{'class':'title'})
    prices=soup.find_all('td',{'class':'price numeric new_price'})
    counter=0
    flag=0
    price_length=len(prices)
    
    if game_year<2006:
        prices_list.append(np.nan)
        #print(name," , ",np.nan)
        
    else:           
        price=avg_price(prices[:price_length],price_length) 
        prices_list.append(price)
        #print(name," , ",price)
          


In [5]:
df=pd.read_csv('fixed_without_price.csv', index_col='index')
names=df['name'].tolist()

console=df['platforms']
dates=df['release year']

flag=0
for i in names:
    if dates[flag] =='TBA - Early Access':
        flag+=1
        prices_list.append(np.nan)
        print('tba')
        continue
    #if flag==100:
        #break
    scrape_each_game_price(i,i.replace(" ", "+"),dates[flag])
    flag+=1
df.insert(1,'price',prices_list)
df.to_csv('fixed_WITH_price.csv')
display(df)


ValueError: cannot insert price, already exists

In [13]:

df.to_csv('fixed_WITH_price.csv')